# Jointure MapReduce

In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Using cached version of Emiasd-Flight-Data-Analysis.jar


error: error while loading FlightDataLoader, class file '/tmp/toree-tmp-dir11599394110815113568/toree_add_jars/Emiasd-Flight-Data-Analysis.jar(com/flightdelay/data/loaders/FlightDataLoader.class)' has location not matching its contents: contains class FlightDataLoader


Using cached version of Emiasd-Flight-Data-Analysis.jar


In [2]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader}
import com.flightdelay.data.loaders.FlightDataLoader

//Env Configuration
val args: Array[String] = Array("juniper")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)

args = Array(juniper)
spark = org.apache.spark.sql.SparkSession@473bfc85
session = org.apache.spark.sql.SparkSession@473bfc85
configuration = AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileCo...


AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileCo...

# Chargement Flight et Weather

In [3]:
// Chargement de Flight Data
val processedFlightPath = s"${configuration.common.output.basePath}/common/data/processed_flights.parquet"
val flightDF = spark.read.parquet(processedFlightPath)

val processedWeatherPath = s"${configuration.common.output.basePath}/common/data/processed_weather.parquet"
val weatherDF = spark.read.parquet(processedWeatherPath)



processedFlightPath = /home/jovyan/work/output/common/data/processed_flights.parquet
flightDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 97 more fields]
processedWeatherPath = /home/jovyan/work/output/common/data/processed_weather.parquet
weatherDF = [WBAN: string, Date: date ... 65 more fields]


[WBAN: string, Date: date ... 65 more fields]

## Weather

In [4]:
import java.sql.Date

val weatherColumns = Seq(
  "WBAN",
  "Date",
  "Time",
  "StationType",
  "SkyCondition",
  "Visibility",
)

val weatherData = weatherDF
  .select(weatherColumns.head, weatherColumns.tail: _*)
  .orderBy("WBAN", "Date", "Time")
weatherData.show(10)

+-----+----------+----+-----------+------------+----------+
| WBAN|      Date|Time|StationType|SkyCondition|Visibility|
+-----+----------+----+-----------+------------+----------+
|03011|2012-01-01|0000|          0|         CLR|      10.0|
|03011|2012-01-01|0100|          0|         CLR|      10.0|
|03011|2012-01-01|0200|          0|         CLR|      10.0|
|03011|2012-01-01|0300|          0|         CLR|      10.0|
|03011|2012-01-01|0400|          0|         CLR|      10.0|
|03011|2012-01-01|0500|          0|         CLR|      10.0|
|03011|2012-01-01|0600|          0|         CLR|      10.0|
|03011|2012-01-01|0700|          0|         CLR|      10.0|
|03011|2012-01-01|0800|          0|         CLR|      10.0|
|03011|2012-01-01|0900|          0|         CLR|      10.0|
+-----+----------+----+-----------+------------+----------+
only showing top 10 rows



weatherColumns = List(WBAN, Date, Time, StationType, SkyCondition, Visibility)
weatherData = [WBAN: string, Date: date ... 4 more fields]


[WBAN: string, Date: date ... 4 more fields]

## Flights

In [6]:
val flightColumns = Seq(
  "UTC_FL_DATE",
  "UTC_CRS_DEP_TIME",
  "FL_DATE",
  "CRS_DEP_TIME",  
  "ORIGIN_WBAN",
  "DEST_WBAN",
  "CRS_ELAPSED_TIME",
  "feature_utc_departure_hour",
  "OP_CARRIER_AIRLINE_ID",
  "OP_CARRIER_FL_NUM",  
  //"feature_flight_month",
  //"feature_flight_year",
  //"feature_flight_quarter",
  "feature_flight_day_of_week",
  "feature_is_weekend",  
  "feature_departure_hour_rounded",
  "feature_utc_departure_hour_rounded",
    
  "label_is_delayed_15min"
)

val flightData = flightDF
  .select(flightColumns.head, flightColumns.tail: _*)

flightData.show(10) 


+-----------+----------------+----------+------------+-----------+---------+----------------+--------------------------+---------------------+-----------------+--------------------------+------------------+------------------------------+----------------------------------+----------------------+
|UTC_FL_DATE|UTC_CRS_DEP_TIME|   FL_DATE|CRS_DEP_TIME|ORIGIN_WBAN|DEST_WBAN|CRS_ELAPSED_TIME|feature_utc_departure_hour|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|feature_flight_day_of_week|feature_is_weekend|feature_departure_hour_rounded|feature_utc_departure_hour_rounded|label_is_delayed_15min|
+-----------+----------------+----------+------------+-----------+---------+----------------+--------------------------+---------------------+-----------------+--------------------------+------------------+------------------------------+----------------------------------+----------------------+
| 2012-01-02|            0155|2012-01-01|        1955|      94846|    23050|           180.0|                   

flightColumns = List(UTC_FL_DATE, UTC_CRS_DEP_TIME, FL_DATE, CRS_DEP_TIME, ORIGIN_WBAN, DEST_WBAN, CRS_ELAPSED_TIME, feature_utc_departure_hour, OP_CARRIER_AIRLINE_ID, OP_CARRIER_FL_NUM, feature_flight_day_of_week, feature_is_weekend, feature_departure_hour_rounded, feature_utc_departure_hour_rounded, label_is_delayed_15min)
flightData = [UTC_FL_DATE: date, UTC_CRS_DEP_TIME: string ... 13 more fields]


[UTC_FL_DATE: date, UTC_CRS_DEP_TIME: string ... 13 more fields]

# Jointure Map Reduce

In [7]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.Row
import scala.collection.JavaConverters._

// Recommandé: travailler en UTC pour éviter les surprises
spark.conf.set("spark.sql.session.timeZone", "UTC")
spark.conf.set("spark.sql.crossJoin.enabled", "true")
spark.conf.set("spark.sql.debug.maxToStringFields", "100000")


In [8]:
// -----------------------------------------
// 0) Normalisation du dataset Weather
//    - DATE  -> date
//    - TIME  -> heure 2 chiffres "HH"
// -----------------------------------------
val weatherNorm = weatherData
  // DATE peut être string => on force en date (UTC)
  .withColumn("W_DATE", to_date(col("DATE")))
  // TIME peut être "0", "00", "0030", 0, etc.
  .withColumn("TIME_STR", col("TIME").cast("string"))
  .withColumn(
    "W_HOUR",
    when(length(col("TIME_STR")) >= 2, substring(col("TIME_STR"), 1, 2))
      .otherwise(lpad(col("TIME_STR"), 2, "0"))
  )
  .select(
    col("WBAN"),
    col("W_DATE"),
    col("W_HOUR"),
    // garde ce qui t'intéresse côté météo :
    col("*") // tu peux remplacer par les colonnes utiles: vent, visib, etc.
  )
weatherNorm.show(12)

+-----+----------+------+-----+----------+----+-----------+------------+----------+----------+--------+------+
| WBAN|    W_DATE|W_HOUR| WBAN|      Date|Time|StationType|SkyCondition|Visibility|    W_DATE|TIME_STR|W_HOUR|
+-----+----------+------+-----+----------+----+-----------+------------+----------+----------+--------+------+
|03011|2012-01-01|    00|03011|2012-01-01|0000|          0|         CLR|      10.0|2012-01-01|    0000|    00|
|03011|2012-01-01|    01|03011|2012-01-01|0100|          0|         CLR|      10.0|2012-01-01|    0100|    01|
|03011|2012-01-01|    02|03011|2012-01-01|0200|          0|         CLR|      10.0|2012-01-01|    0200|    02|
|03011|2012-01-01|    03|03011|2012-01-01|0300|          0|         CLR|      10.0|2012-01-01|    0300|    03|
|03011|2012-01-01|    04|03011|2012-01-01|0400|          0|         CLR|      10.0|2012-01-01|    0400|    04|
|03011|2012-01-01|    05|03011|2012-01-01|0500|          0|         CLR|      10.0|2012-01-01|    0500|    05|
|

weatherNorm = [WBAN: string, W_DATE: date ... 10 more fields]


[WBAN: string, W_DATE: date ... 10 more fields]

In [9]:
// Option: pour le shuffle, tu peux aider Spark à partitionner:
val weatherPart = weatherNorm.repartition(col("WBAN"), col("W_DATE"))

weatherPart = [WBAN: string, W_DATE: date ... 10 more fields]


[WBAN: string, W_DATE: date ... 10 more fields]

In [10]:
// ───────────────────────────────────────────────────────────────────────────────
// 0) Crée un identifiant de vol déterministe "flight_id"
//    (tu peux remplacer par monotonically_increasing_id() si tu préfères)
// ───────────────────────────────────────────────────────────────────────────────
val flights0 = flightData
  .withColumn(
    "flight_id",
    concat_ws("_",
      date_format(col("UTC_FL_DATE").cast("date"), "yyyyMMdd"),
      col("OP_CARRIER_AIRLINE_ID").cast("string"),
      col("OP_CARRIER_FL_NUM").cast("string"),
      col("ORIGIN_WBAN").cast("string"),
      col("DEST_WBAN").cast("string"),
      date_format(col("UTC_CRS_DEP_TIME"), "HHmm")
    )
  )

flights0.show(10)

flights0 = [UTC_FL_DATE: date, UTC_CRS_DEP_TIME: string ... 14 more fields]


+-----------+----------------+----------+------------+-----------+---------+----------------+--------------------------+---------------------+-----------------+--------------------------+------------------+------------------------------+----------------------------------+----------------------+--------------------+
|UTC_FL_DATE|UTC_CRS_DEP_TIME|   FL_DATE|CRS_DEP_TIME|ORIGIN_WBAN|DEST_WBAN|CRS_ELAPSED_TIME|feature_utc_departure_hour|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|feature_flight_day_of_week|feature_is_weekend|feature_departure_hour_rounded|feature_utc_departure_hour_rounded|label_is_delayed_15min|           flight_id|
+-----------+----------------+----------+------------+-----------+---------+----------------+--------------------------+---------------------+-----------------+--------------------------+------------------+------------------------------+----------------------------------+----------------------+--------------------+
| 2012-01-02|            0155|2012-01-01|        

[UTC_FL_DATE: date, UTC_CRS_DEP_TIME: string ... 14 more fields]

In [11]:

// ───────────────────────────────────────────────────────────────────────────────
// 1) Base temporelle départ (UTC) : "yyyy-MM-dd HH" → epoch (secondes)
// ───────────────────────────────────────────────────────────────────────────────
val flightsBase = flights0
  .withColumn(
    "FL_DATE_STR",
    when(col("UTC_FL_DATE").cast("date").isNotNull,
         date_format(col("UTC_FL_DATE").cast("date"), "yyyy-MM-dd"))
    .otherwise(col("UTC_FL_DATE").cast("string"))
  )
  .withColumn("DEP_HOUR_2D", lpad(col("feature_utc_departure_hour_rounded").cast("string"), 2, "0"))
  .withColumn("base_epoch",
    unix_timestamp(concat_ws(" ", col("FL_DATE_STR"), col("DEP_HOUR_2D")), "yyyy-MM-dd HH")
  )
  // IMPORTANT : on ne garde QUE ce qu'on va utiliser ensuite (dont flight_id !)
  .select(
    col("flight_id"),
    col("ORIGIN_WBAN"),
    col("DEST_WBAN"),
    col("FL_DATE_STR"),
    col("DEP_HOUR_2D"),
    col("base_epoch"),
    col("label_is_delayed_15min")  
  )

flightsBase.show(10)

+--------------------+-----------+---------+-----------+-----------+----------+----------------------+
|           flight_id|ORIGIN_WBAN|DEST_WBAN|FL_DATE_STR|DEP_HOUR_2D|base_epoch|label_is_delayed_15min|
+--------------------+-----------+---------+-----------+-----------+----------+----------------------+
|20120102_20398_38...|      94846|    23050| 2012-01-02|         20|1325534400|                     1|
|20120113_20398_38...|      94846|    23050| 2012-01-13|         20|1326484800|                     1|
|20120127_20398_37...|      94846|    23050| 2012-01-27|         16|1327680000|                     1|
|20120118_20398_38...|      94846|    23050| 2012-01-18|         20|1326916800|                     1|
|20120123_20398_37...|      94846|    23050| 2012-01-23|         16|1327334400|                     1|
|20120121_20398_37...|      94846|    23050| 2012-01-21|         16|1327161600|                     1|
|20120123_20398_38...|      94846|    23050| 2012-01-23|         20|13273

flightsBase = [flight_id: string, ORIGIN_WBAN: string ... 5 more fields]


[flight_id: string, ORIGIN_WBAN: string ... 5 more fields]

In [12]:
// ───────────────────────────────────────────────────────────────────────────────
// 2) Générer les 12 heures cibles t-1h..t-12h (crossJoin offsets 1..12)
// ───────────────────────────────────────────────────────────────────────────────
val offsets = flightsBase.sparkSession.range(1, 13).toDF("h")  // 1..12

val originTargets = flightsBase
  .crossJoin(offsets)
  .withColumn("target_epoch", col("base_epoch") - col("h") * lit(3600))
  .withColumn("TARGET_DATE", to_date(from_unixtime(col("target_epoch"))))
  .withColumn("TARGET_HOUR_2D", date_format(from_unixtime(col("target_epoch")), "HH"))
  .select(
    col("flight_id"),                                    // ← remplace flightId par flight_id
    col("ORIGIN_WBAN").as("WBAN_KEY"),
    col("h").cast("int").as("o_h"),
    col("TARGET_DATE"),
    col("TARGET_HOUR_2D"),
    col("label_is_delayed_15min")  
  )

// (debug)
originTargets.show(12, truncate = false)

+------------------------------------+--------+---+-----------+--------------+----------------------+
|flight_id                           |WBAN_KEY|o_h|TARGET_DATE|TARGET_HOUR_2D|label_is_delayed_15min|
+------------------------------------+--------+---+-----------+--------------+----------------------+
|20120102_20398_3864_94846_23050_0000|94846   |1  |2012-01-02 |19            |1                     |
|20120102_20398_3864_94846_23050_0000|94846   |2  |2012-01-02 |18            |1                     |
|20120102_20398_3864_94846_23050_0000|94846   |3  |2012-01-02 |17            |1                     |
|20120102_20398_3864_94846_23050_0000|94846   |4  |2012-01-02 |16            |1                     |
|20120102_20398_3864_94846_23050_0000|94846   |5  |2012-01-02 |15            |1                     |
|20120102_20398_3864_94846_23050_0000|94846   |6  |2012-01-02 |14            |1                     |
|20120102_20398_3864_94846_23050_0000|94846   |7  |2012-01-02 |13            |1   

offsets = [h: bigint]
originTargets = [flight_id: string, WBAN_KEY: string ... 4 more fields]


[flight_id: string, WBAN_KEY: string ... 4 more fields]

In [13]:
// ───────────────────────────────────────────────────────────────────────────────
// 3) Normalisation météo (si pas déjà fait) et jointure exacte (WBAN, DATE, HOUR)
// ───────────────────────────────────────────────────────────────────────────────

val originJoined = originTargets
  .join(
    weatherNorm,
    originTargets.col("WBAN_KEY") === weatherNorm.col("WBAN") &&
    originTargets.col("TARGET_DATE") === weatherNorm.col("W_DATE") &&
    originTargets.col("TARGET_HOUR_2D") === weatherNorm.col("W_HOUR"),
    "inner"
  )
  .select(
    col("flight_id"),
    col("o_h"),
    col("WBAN_KEY").as("WBAN"),
    col("TARGET_DATE").as("o_target_date"),
    col("TARGET_HOUR_2D").as("o_target_hour"),
    col("W_DATE"), col("W_HOUR"),
    col("DATE"), col("TIME"),
    col("StationType"),
    col("SkyCondition"),
    col("Visibility"),
    //col("feature_ceiling"),  
  ).orderBy(col("flight_id"), col("o_target_hour"))

originJoined.show(20, false)

+------------------------------------+---+-----+-------------+-------------+----------+------+----------+----+-----------+------------+----------+
|flight_id                           |o_h|WBAN |o_target_date|o_target_hour|W_DATE    |W_HOUR|DATE      |TIME|StationType|SkyCondition|Visibility|
+------------------------------------+---+-----+-------------+-------------+----------+------+----------+----+-----------+------------+----------+
|20120101_19393_1021_13968_12918_0000|12 |13968|2012-01-01   |06           |2012-01-01|06    |2012-01-01|0600|11         |CLR         |10.0      |
|20120101_19393_1021_13968_12918_0000|11 |13968|2012-01-01   |07           |2012-01-01|07    |2012-01-01|0700|11         |CLR         |10.0      |
|20120101_19393_1021_13968_12918_0000|10 |13968|2012-01-01   |08           |2012-01-01|08    |2012-01-01|0800|11         |CLR         |10.0      |
|20120101_19393_1021_13968_12918_0000|9  |13968|2012-01-01   |09           |2012-01-01|09    |2012-01-01|0900|11      

originJoined = [flight_id: string, o_h: int ... 10 more fields]


[flight_id: string, o_h: int ... 10 more fields]

In [14]:
// ───────────────────────────────────────────────────────────────────────────────
// 4) (Option) Regrouper en liste par vol (gardez o_h pour trier côté lecteur)
// ───────────────────────────────────────────────────────────────────────────────
val originAgg = originJoined
  .withColumn("o_pack", struct(
    col("o_h"),
    col("o_target_date"),
    col("o_target_hour"),
    col("W_DATE"),
    col("W_HOUR"),
    col("SkyCondition"),
    col("Visibility"),
    // + tes variables météo utiles
  ))
  .groupBy("flight_id")                       
  .agg(collect_list(col("o_pack")).as("W_o_list"))

originAgg.show(10, 500, false)

+------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                           flight_id|                                                                                                                                                                                                                                                                                                                                                                                                                                      

originAgg = [flight_id: string, W_o_list: array<struct<o_h:int,o_target_date:date,o_target_hour:string,W_DATE:date,W_HOUR:string,SkyCondition:string,Visibility:double>>]


[flight_id: string, W_o_list: array<struct<o_h:int,o_target_date:date,o_target_hour:string,W_DATE:date,W_HOUR:string,SkyCondition:string,Visibility:double>>]

# Jointure Map Reduce 2

In [4]:
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

object SimpleFlightWeatherJoin {

  /**
   * Jointure spatio-temporelle entre vols et observations météo
   * Implémentation MapReduce de l'algorithme du paper
   * 
   * @param flightDF DataFrame des vols avec colonnes: FL_DATE, CRS_DEP_TIME, ORIGIN_WBAN, etc.
   * @param weatherDF DataFrame météo avec colonnes: WBAN, Date, Time, DryBulbFarenheit, etc.
   * @return DataFrame jointé avec structure: [flight_data, weather_observations(array)]
   */
  def joinFlightsWithWeather(
    flightDF: DataFrame, 
    weatherDF: DataFrame
  ): DataFrame = {
    
    import flightDF.sparkSession.implicits._
    
    // ============================================
    // PHASE MAP - Préparation des données
    // ============================================
    
    // MAP Weather (OT): Tagger les observations météo
    val weatherTagged = weatherDF
      .withColumn("table_tag", lit("OT"))
      .withColumn("join_key_wban", col("WBAN"))
      .withColumn("join_key_date", col("Date"))
    
    // MAP Flights (FT): Tagger les vols
    // Arrondir l'heure de départ vers le bas (floor)
    val flightTagged = flightDF
      .withColumn("table_tag", lit("FT"))
      .withColumn("join_key_wban", col("ORIGIN_WBAN"))
      .withColumn("join_key_date", col("FL_DATE"))
      .withColumn("dep_hour_rounded", floor(col("CRS_DEP_TIME") / 100) * 100)
    
    // Dupliquer les vols de fin de journée pour le jour suivant
    // (si départ + 12h dépasse minuit)
    val flightDuplicated = flightTagged
      .withColumn("needs_next_day", 
        when(col("dep_hour_rounded") / 100 + 12 >= 24, 1).otherwise(0))
      .withColumn("duplicate_count", 
        when(col("needs_next_day") === 1, array(lit(0), lit(1))).otherwise(array(lit(0))))
      .withColumn("day_offset", explode(col("duplicate_count")))
      .withColumn("join_key_date", 
        when(col("day_offset") === 1, date_add(col("join_key_date"), 1))
        .otherwise(col("join_key_date")))
      .drop("needs_next_day", "duplicate_count", "day_offset")
    
    // ============================================
    // PHASE SHUFFLE - Union et Partitionnement
    // ============================================
    
    // Union des deux datasets avec composite_key
    val weatherForJoin = weatherTagged
      .select(
        col("join_key_wban").as("key_wban"),
        col("join_key_date").as("key_date"),
        col("table_tag"),
        col("WBAN"),
        col("Date"),
        col("Time").cast("int").as("Time"), // Convertir Time en INT
        col("StationType"),
        col("SkyCondition"),
        col("Visibility"),
        col("DryBulbFarenheit"),
        col("DryBulbCelsius"),
        col("WindSpeed"),
        col("WindDirection"),
        col("RelativeHumidity")
      )
    
    val flightForJoin = flightDuplicated
      .select(
        col("join_key_wban").as("key_wban"),
        col("join_key_date").as("key_date"),
        col("table_tag"),
        col("UTC_FL_DATE"),
        col("FL_DATE"),
        col("CRS_DEP_TIME"),
        col("dep_hour_rounded"),
        col("ORIGIN_WBAN"),
        col("DEST_WBAN"),
        col("CRS_ELAPSED_TIME"),
        col("OP_CARRIER_AIRLINE_ID"),
        col("OP_CARRIER_FL_NUM"),
        col("label_is_delayed_15min")
      )
    
    // ============================================
    // PHASE REDUCE - Jointure et Agrégation
    // ============================================
    
    // Créer une fenêtre de partitionnement par (WBAN, Date)
    val windowSpec = Window
      .partitionBy("key_wban", "key_date")
      .orderBy("Time")
    
    // Collecter toutes les observations météo par (WBAN, Date)
    val weatherGrouped = weatherForJoin
      .groupBy("key_wban", "key_date")
      .agg(
        collect_list(
          struct(
            col("WBAN"),
            col("Date"),
            col("Time"), // Déjà en INT grâce au cast précédent
            col("StationType"),
            col("SkyCondition"),
            col("Visibility"),
            col("DryBulbFarenheit"),
            col("DryBulbCelsius"),
            col("WindSpeed"),
            col("WindDirection"),
            col("RelativeHumidity")
          )
        ).as("all_weather_obs")
      )
    
    // Jointure avec les vols
    val joined = flightForJoin
      .join(
        weatherGrouped,
        flightForJoin("key_wban") === weatherGrouped("key_wban") &&
        flightForJoin("key_date") === weatherGrouped("key_date"),
        "left"
      )
    
    // Filtrer les observations pour ne garder que les 13 heures nécessaires
    // (de -12h à 0h par rapport à l'heure de départ arrondie)
    val joinedWithFiltered = joined
      .withColumn("required_hours", 
        expr("""
          transform(
            sequence(0, 12),
            i -> cast((dep_hour_rounded / 100 - (12 - i) + 24) % 24 * 100 as int)
          )
        """)
      )
      .withColumn("weather_observations",
        expr("""
          filter(
            all_weather_obs,
            obs -> array_contains(required_hours, obs.Time)
          )
        """)
      )
    
    // Sélectionner les colonnes finales
    val result = joinedWithFiltered
      .select(
        // Informations du vol
        col("UTC_FL_DATE"),
        col("FL_DATE"),
        col("CRS_DEP_TIME"),
        col("dep_hour_rounded"),
        col("ORIGIN_WBAN"),
        col("DEST_WBAN"),
        col("CRS_ELAPSED_TIME"),
        col("OP_CARRIER_AIRLINE_ID"),
        col("OP_CARRIER_FL_NUM"),
        col("label_is_delayed_15min"),
        
        // Observations météo (array)
        col("weather_observations")
      )
      .filter(size(col("weather_observations")) > 0) // Garder uniquement les vols avec météo
    
    result
  }

}

val joinedDF = SimpleFlightWeatherJoin.joinFlightsWithWeather(flightDF, weatherDF)
joinedDF.show(10, 10000, true)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

defined object SimpleFlightWeatherJoin
joinedDF = [UTC_FL_DATE: date, FL_DATE: date ... 9 more fields]


[UTC_FL_DATE: date, FL_DATE: date ... 9 more fields]